# Task B - single subject - model ?

## Notebook setup
This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- label: index of feature column to be selected to perform activity detection, between [0,6]. The default value for task B is 6;
- folder: directory name where '.mat' files are stored;
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
label = 0
folder = "../data/full/"
window_size = 15
stride = 5

In [2]:
import preprocessing
import models
import utils
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
if not(os.path.exists("./data")):
    os.mkdir("./data")

# One-shot classification
Here classification is performed with null class.
### Preprocessing

In [4]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=False,
                                                                                                        null_class=True,
                                                                                                        print_info=False)


Processing data from subject 1

Processing data from subject 2

Processing data from subject 3


Y_train and Y_test contain the correct labels for each signals window. Y_test in particular will be used to evaluate predictions for both this (one-shot) and the two-steps models. For this reason it is here saved with a different name, to avoid having it being overwritten later.

In [5]:
Y_test_true = Y_test

### Model

In [6]:
oneshot_model = models.Convolutional((window_size, n_features), n_classes, print_info=False)

oneshot_model.compile(optimizer = Adam(lr=0.001),
                      loss = "categorical_crossentropy", 
                      metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_AOS_mult.hdf5', verbose=1, save_best_only=True)

### Training

In [7]:
oneshot_model.fit(x = X_train, 
                  y = to_categorical(Y_train),
                  epochs = 15,
                  batch_size = 128,
                  verbose = 1,
                  callbacks=[checkpointer],
                  validation_data=(X_test, to_categorical(Y_test)),
                  class_weight=class_weights)

Train on 59176 samples, validate on 21341 samples
Epoch 1/15
59176/59176 [==============================] - 10s 176us/step - loss: 0.5885 - acc: 0.7851 - val_loss: 0.9146 - val_acc: 0.7602

Epoch 00001: val_loss improved from inf to 0.91461, saving model to ./data/model_AOS_mult.hdf5
Epoch 2/15
59176/59176 [==============================] - 6s 109us/step - loss: 0.4143 - acc: 0.8498 - val_loss: 0.8840 - val_acc: 0.7651

Epoch 00002: val_loss improved from 0.91461 to 0.88399, saving model to ./data/model_AOS_mult.hdf5
Epoch 3/15
59176/59176 [==============================] - 6s 109us/step - loss: 0.3692 - acc: 0.8674 - val_loss: 0.9768 - val_acc: 0.7730

Epoch 00003: val_loss did not improve
Epoch 4/15
59176/59176 [==============================] - 6s 110us/step - loss: 0.3438 - acc: 0.8743 - val_loss: 0.9817 - val_acc: 0.7757

Epoch 00004: val_loss did not improve
Epoch 5/15
59176/59176 [==============================] - 7s 111us/step - loss: 0.3279 - acc: 0.8807 - val_loss: 0.9719 - v

### Evaluation - passare class_weights a class report

In [8]:
Y_pred = oneshot_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.81      0.55      0.66      2475
          1       0.74      0.93      0.83      7465
          2       0.80      0.75      0.77      5445
          3       0.86      0.72      0.78      4846
          4       0.94      0.92      0.93      1110

avg / total       0.80      0.79      0.79     21341

Weighted f1-score: 0.7872147643014269


In [9]:
oneshot_model_best = load_model('./data/model_AOS_mult.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.72      0.60      0.66      2475
          1       0.73      0.95      0.83      7465
          2       0.82      0.69      0.75      5445
          3       0.87      0.71      0.78      4846
          4       0.93      0.89      0.91      1110

avg / total       0.79      0.79      0.78     21341

Weighted f1-score: 0.7813867327367077


# Two-steps classification
## Activity detection
This model performs a binary classification.
### Preprocessing

In [10]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=True,
                                                                                                        null_class=True,
                                                                                                        print_info=False)


Processing data from subject 1

Processing data from subject 2

Processing data from subject 3


### Model

In [11]:
detection_model = models.ConvolutionalRecurrent((window_size, n_features), n_classes, print_info=False)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_ATSD_mult.hdf5', verbose=1, save_best_only=True)

### Training

In [12]:
detection_model.fit(x = X_train, 
                    y = to_categorical(Y_train), 
                    epochs = 15, 
                    batch_size = 128,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, to_categorical(Y_test)),
                    class_weight=class_weights)

Train on 59176 samples, validate on 21341 samples
Epoch 1/15
59176/59176 [==============================] - 14s 236us/step - loss: 0.1156 - acc: 0.9605 - val_loss: 0.2127 - val_acc: 0.9265

Epoch 00001: val_loss improved from inf to 0.21267, saving model to ./data/model_ATSD_mult.hdf5
Epoch 2/15
59176/59176 [==============================] - 11s 178us/step - loss: 0.0805 - acc: 0.9724 - val_loss: 0.2321 - val_acc: 0.9238

Epoch 00002: val_loss did not improve
Epoch 3/15
59176/59176 [==============================] - 11s 177us/step - loss: 0.0714 - acc: 0.9764 - val_loss: 0.2298 - val_acc: 0.9236

Epoch 00003: val_loss did not improve
Epoch 4/15
59176/59176 [==============================] - 11s 177us/step - loss: 0.0658 - acc: 0.9783 - val_loss: 0.2317 - val_acc: 0.9263

Epoch 00004: val_loss did not improve
Epoch 5/15
59176/59176 [==============================] - 11s 180us/step - loss: 0.0609 - acc: 0.9802 - val_loss: 0.2543 - val_acc: 0.9271

Epoch 00005: val_loss did not improve
Ep

### Evaluation

In [13]:
Y_pred = detection_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.76      0.61      0.67      2475
          1       0.95      0.98      0.96     18866

avg / total       0.93      0.93      0.93     21341

Weighted f1-score: 0.9288907890886872


In [14]:
detection_model_best = load_model('./data/model_ATSD_mult.hdf5')

Y_pred = detection_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.76      0.54      0.63      2475
          1       0.94      0.98      0.96     18866

avg / total       0.92      0.93      0.92     21341

Weighted f1-score: 0.9209569053855828


In [15]:
Y_pred_d = Y_pred

## Activity classification

In [16]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=False,
                                                                                                        null_class=False,
                                                                                                        print_info=False)


Processing data from subject 1

Processing data from subject 2

Processing data from subject 3


### Model

In [17]:
classification_model = models.ConvolutionalRecurrent((window_size, n_features), n_classes, print_info=False)

classification_model.compile(optimizer = Adam(lr=0.001),
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_ATSC_mult.hdf5', verbose=1, save_best_only=True)

### Training

In [18]:
classification_model.fit(x = X_train,
                         y = to_categorical(Y_train), 
                         epochs = 15, 
                         batch_size = 128,
                         verbose = 1,
                         callbacks=[checkpointer],
                         validation_data=(X_test, to_categorical(Y_test)),
                         class_weight=class_weights)

Train on 55852 samples, validate on 18866 samples
Epoch 1/15
55852/55852 [==============================] - 13s 237us/step - loss: 0.3996 - acc: 0.8448 - val_loss: 0.7824 - val_acc: 0.8180

Epoch 00001: val_loss improved from inf to 0.78243, saving model to ./data/model_ATSC_mult.hdf5
Epoch 2/15
55852/55852 [==============================] - 10s 174us/step - loss: 0.2724 - acc: 0.8966 - val_loss: 0.6166 - val_acc: 0.8297

Epoch 00002: val_loss improved from 0.78243 to 0.61662, saving model to ./data/model_ATSC_mult.hdf5
Epoch 3/15
55852/55852 [==============================] - 10s 174us/step - loss: 0.2506 - acc: 0.9035 - val_loss: 0.7486 - val_acc: 0.8333

Epoch 00003: val_loss did not improve
Epoch 4/15
55852/55852 [==============================] - 10s 174us/step - loss: 0.2348 - acc: 0.9103 - val_loss: 0.5396 - val_acc: 0.8336

Epoch 00004: val_loss improved from 0.61662 to 0.53957, saving model to ./data/model_ATSC_mult.hdf5
Epoch 5/15
55852/55852 [==============================] 

### Evaluation

In [19]:
Y_pred = classification_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.73      0.95      0.82      7465
          1       0.94      0.76      0.84      5445
          2       0.93      0.72      0.81      4846
          3       0.98      0.92      0.94      1110

avg / total       0.86      0.83      0.83     18866

Weighted f1-score: 0.832140837308565


In [20]:
classification_model_best = load_model('./data/model_ATSC_mult.hdf5')

Y_pred = classification_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.75      0.90      0.82      7465
          1       0.87      0.84      0.85      5445
          2       0.96      0.71      0.82      4846
          3       0.95      0.93      0.94      1110

avg / total       0.85      0.83      0.83     18866

Weighted f1-score: 0.8338328873568304


## Cascade of detection and classification
The labels that have to be used for assessment are saved in Y_test_true. The labels predicted by the detection_model are saved instead in Y_pred_d.

In [21]:
print(Y_test_true.shape, Y_pred_d.shape)

(21341,) (21341,)


In [22]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=True,
                                                                                                        null_class=True,
                                                                                                        print_info=False)


Processing data from subject 1

Processing data from subject 2

Processing data from subject 3


In [23]:
mask = (Y_pred_d == 1)
X_detected = X_test[mask, :, :]
Y_pred_c = classification_model_best.predict_classes(X_detected)
Y_pred_d[mask] = Y_pred_c

In [24]:
print(classification_report(Y_test_true, Y_pred_d))
print("Weighted f1-score:", f1_score(Y_test_true, Y_pred_d, average='weighted'))

             precision    recall  f1-score   support

          0       0.13      0.56      0.21      2475
          1       0.12      0.09      0.10      7465
          2       0.01      0.00      0.01      5445
          3       0.02      0.01      0.01      4846
          4       0.00      0.00      0.00      1110

avg / total       0.06      0.10      0.06     21341

Weighted f1-score: 0.06404905198508254


C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


One-shot classification instead had:

In [25]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=False,
                                                                                                        null_class=True,
                                                                                                        print_info=False)


Processing data from subject 1

Processing data from subject 2

Processing data from subject 3


In [26]:
oneshot_model_best = load_model('./data/model_AOS_mult.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))
print("Weighted f1-score:", f1_score(Y_test, Y_pred, average='weighted'))

             precision    recall  f1-score   support

          0       0.72      0.60      0.66      2475
          1       0.73      0.95      0.83      7465
          2       0.82      0.69      0.75      5445
          3       0.87      0.71      0.78      4846
          4       0.93      0.89      0.91      1110

avg / total       0.79      0.79      0.78     21341

Weighted f1-score: 0.7813867327367077


# end